In [1]:
import numpy as np
import pandas as pd

from Utils.data_proccessing import get_historical_ohlc
from pybit.unified_trading import HTTP
from constants import API_KEY, API_SECRET
from Utils.plotting import plotting_chart_technicals

In [2]:
session = HTTP(demo=True, api_key=API_KEY, api_secret=API_SECRET)
ohlc_m15 = get_historical_ohlc(session=session, symbol='BTCUSDT', interval='15')
ohlc_m5 = get_historical_ohlc(session=session, symbol='BTCUSDT', interval='5')

In [3]:
def detect_fvg(last_3_candels):

    fvg = 0
    fvg_top = 0
    fvg_bottom = 0

    high = last_3_candels['high'].values
    low = last_3_candels['low'].values

    if low[-1] > high[0]:

        fvg = 1
        fvg_top = low[-1]
        fvg_bottom = high[0]

    elif high[-1] < low[0]:
        
        fvg = -1
        fvg_top = low[0]
        fvg_bottom = high[-1]

    return fvg, fvg_top, fvg_bottom

In [4]:
def detect_fractal(last_3_candels):
    
    fractal = 0
    fractal_level = 0

    high = last_3_candels['high'].values
    low = last_3_candels['low'].values

    if high[-2] > high[-1] and high[-2] > high[-3]:

        fractal += 1
        fractal_level = high[-2]

    if low[-2] < low[-1] and low[-2] < low[-3]:

        fractal -= 1
        fractal_level = low[-2]

    return fractal, fractal_level

In [5]:
all_fractal = [0,0]
all_fractal_levels = [0,0]

for i in range(2, len(ohlc_m5)):
    
    last_3_candels = ohlc_m5.iloc[i-2:i+1]
    fractal, fractal_level = detect_fractal(last_3_candels)
    all_fractal.append(fractal)
    all_fractal_levels.append(fractal_level)

all_fractal = pd.Series(all_fractal, name='fractal')
all_fractal_levels = pd.Series(all_fractal_levels, name='fractal_level')

ohlc_m5 = pd.concat([ohlc_m5, all_fractal, all_fractal_levels], axis=1)

In [6]:
all_fvg = [0,0]
all_fvg_top = [0,0]
all_fvg_bottom = [0,0]

for i in range(2, len(ohlc_m15)):
    
    last_3_candels = ohlc_m15.iloc[i-2:i+1]
    fvg, fvg_top, fvg_bottom = detect_fvg(last_3_candels)
    all_fvg.append(fvg)
    all_fvg_top.append(fvg_top)
    all_fvg_bottom.append(fvg_bottom)

all_fvg = pd.Series(all_fvg, name='fvg')
all_fvg_top = pd.Series(all_fvg_top, name='fvg_top')
all_fvg_bottom = pd.Series(all_fvg_bottom, name='fvg_bottom')


ohlc_m15 = pd.concat([ohlc_m15, all_fvg, all_fvg_top, all_fvg_bottom], axis=1)

In [7]:
ohlc_m5.tail(10)

,time,open,high,low,close,volume,fractal,fractal_level
2490,2025-10-07 14:45:00,122778.5,122969.0,122734.0,122796.8,683.813,-1,122523.7
2491,2025-10-07 14:50:00,122796.8,122796.8,122530.0,122630.1,506.688,1,122969.0
2492,2025-10-07 14:55:00,122630.1,122764.1,122553.8,122716.0,555.608,-1,122530.0
2493,2025-10-07 15:00:00,122716.0,122841.6,122550.0,122600.0,649.924,0,0.0
2494,2025-10-07 15:05:00,122600.0,122600.1,121650.0,121740.0,4628.140,1,122841.6
2495,2025-10-07 15:10:00,121740.0,122226.1,121650.0,121919.7,1722.122,0,0.0
2496,2025-10-07 15:15:00,121919.7,122337.7,121818.2,122149.9,1046.942,0,0.0
2497,2025-10-07 15:20:00,122149.9,122365.4,122149.9,122299.7,659.657,0,0.0
2498,2025-10-07 15:25:00,122299.7,122330.5,122027.2,122057.2,552.909,1,122365.4
2499,2025-10-07 15:30:00,122057.2,122124.2,121900.0,121913.6,329.285,0,0.0


In [8]:
ohlc_m15.tail(10)

,time,open,high,low,close,volume,fvg,fvg_top,fvg_bottom
2490,2025-10-07 13:15:00,124687.3,124778.9,124569.5,124756.6,532.327,0,0.0,0.0
2491,2025-10-07 13:30:00,124756.6,124762.4,123544.8,123734.2,3545.879,0,0.0,0.0
2492,2025-10-07 13:45:00,123734.2,123940.3,123316.9,123845.7,4274.619,-1,124569.5,123940.3
2493,2025-10-07 14:00:00,123845.7,123962.8,122677.9,122780.1,4965.623,0,0.0,0.0
2494,2025-10-07 14:15:00,122780.1,123298.2,122726.8,122854.3,2646.416,-1,123316.9,123298.2
2495,2025-10-07 14:30:00,122854.3,123000.0,122523.7,122778.5,2374.825,0,0.0,0.0
2496,2025-10-07 14:45:00,122778.5,122969.0,122530.0,122716.0,1746.109,0,0.0,0.0
2497,2025-10-07 15:00:00,122716.0,122841.6,121650.0,121919.7,7000.186,0,0.0,0.0
2498,2025-10-07 15:15:00,121919.7,122365.4,121818.2,122057.2,2259.508,-1,122530.0,122365.4
2499,2025-10-07 15:30:00,122057.2,122124.2,121900.0,121925.6,325.741,0,0.0,0.0


In [9]:
def find_bos_from_fractals(df, mode: str = 'close', max_lookahead=len(ohlc_m5)):
    """
    Повертає копію df з додатковими колонками для рядків-фракталів:
      - fractal_price    : значення fractal_level у рядку фракталу
      - bos_close_price  : close свічки, що зробила BOS (NaN якщо не знайдено)
      - bos_index        : індекс (label) цієї свічки (NaN якщо не знайдено)
      - bos_time         : time цієї свічки (NaN якщо не знайдено)

    mode:
      - 'close' : вважаємо BOS коли close > fractal_level (для fractal==1)
                  або close < fractal_level (для fractal==-1)
      - 'body'  : вважаємо BOS коли мін(open,close) > fractal_level (для fractal==1)
                  або макс(open,close) < fractal_level (для fractal==-1)
    max_lookahead: максимальна кількість свічок у майбутнє (None -> до кінця)
    """
    df = df.copy()

    df['fractal_price'] = np.nan
    df['bos_close_price'] = np.nan
    df['bos_index'] = np.nan
    df['bos_time'] = pd.NaT

    # Індекси де є фрактали (і fractal_level не NaN)
    fractal_mask = df['fractal'].notna() & df['fractal_level'].notna()
    fractal_idxs = list(df.index[fractal_mask])

    pos_of_index = {idx: pos for pos, idx in enumerate(df.index)}
    n = len(df)

    for idx in fractal_idxs:
        f = df.at[idx, 'fractal']
        lvl = df.at[idx, 'fractal_level']
        df.at[idx, 'fractal_price'] = lvl

        start_pos = pos_of_index[idx] + 1
        if max_lookahead is None:
            end_pos = n
        else:
            end_pos = min(n, start_pos + int(max_lookahead))

        # Проходимо по майбутніх свічках
        found = False
        for pos in range(start_pos, end_pos):
            j = df.index[pos]
            o = float(df.iat[pos, df.columns.get_loc('open')])
            c = float(df.iat[pos, df.columns.get_loc('close')])

            if f == 1:
                if mode == 'close':
                    cond = c > lvl
                elif mode == 'body':
                    cond = min(o, c) > lvl
                else:
                    raise ValueError("mode must be 'close' or 'body'")
            elif f == -1:
                if mode == 'close':
                    cond = c < lvl
                elif mode == 'body':
                    cond = max(o, c) < lvl
                else:
                    raise ValueError("mode must be 'close' or 'body'")
            else:
                cond = False

            if cond:
                df.at[idx, 'bos_close_price'] = c
                df.at[idx, 'bos_index'] = j
                df.at[idx, 'bos_time'] = df.at[j, 'time'] if 'time' in df.columns else pd.NaT
                found = True
                break
    return df

ohlc_m5 = find_bos_from_fractals(ohlc_m5, mode='close', max_lookahead=len(ohlc_m5))
ohlc_m5.tail(15)

,time,open,high,low,close,volume,fractal,fractal_level,fractal_price,bos_close_price,bos_index,bos_time
2485,2025-10-07 14:20:00,122959.9,123298.2,122911.0,123079.2,1259.306,0,0.0,0.0,NaN,NaN,NaT
2486,2025-10-07 14:25:00,123079.2,123099.8,122804.0,122854.3,582.057,1,123298.2,123298.2,NaN,NaN,NaT
2487,2025-10-07 14:30:00,122854.3,123000.0,122656.2,122978.2,944.880,0,0.0,0.0,NaN,NaN,NaT
2488,2025-10-07 14:35:00,122978.2,122982.3,122608.9,122682.4,840.096,0,0.0,0.0,NaN,NaN,NaT
2489,2025-10-07 14:40:00,122682.4,122781.1,122523.7,122778.5,589.849,0,0.0,0.0,NaN,NaN,NaT
2490,2025-10-07 14:45:00,122778.5,122969.0,122734.0,122796.8,683.813,-1,122523.7,122523.7,121740.0,2494.0,2025-10-07 15:05:00
2491,2025-10-07 14:50:00,122796.8,122796.8,122530.0,122630.1,506.688,1,122969.0,122969.0,NaN,NaN,NaT
2492,2025-10-07 14:55:00,122630.1,122764.1,122553.8,122716.0,555.608,-1,122530.0,122530.0,121740.0,2494.0,2025-10-07 15:05:00
2493,2025-10-07 15:00:00,122716.0,122841.6,122550.0,122600.0,649.924,0,0.0,0.0,NaN,NaN,NaT
2494,2025-10-07 15:05:00,122600.0,122600.1,121650.0,121740.0,4628.140,1,122841.6,122841.6,NaN,NaN,NaT


In [10]:
def find_fvg_fractal_bos_by_close(ohlc_m15, ohlc_m5, max_lookahead, debug: bool = False):
    """
    Шукає пари фракталів після M15 FVG і першу M5-свічку ПО ЧАСУ після f2,
    у якої close > f1_price (long) або close < f1_price (short).
    Повертає (signals_df, m5_out) — signals і m5 з мітками (в оригінальному індексі).
    """

    m15 = ohlc_m15.copy()
    m5_orig = ohlc_m5.copy()
    m15['time'] = pd.to_datetime(m15['time'])
    m5_orig['time'] = pd.to_datetime(m5_orig['time'])

    m5_orig = m5_orig.reset_index().rename(columns={'index': '_orig_index'})

    # Робоча версія з позиційним індексом 0..N-1
    m5 = m5_orig.sort_values('time').reset_index(drop=True)

    m5['is_f1'] = 0
    m5['is_f2'] = 0
    m5['is_bos'] = 0
    m5['source_fvg_time'] = pd.NaT

    signals = []
    n_m5 = len(m5)

    for _, fvg in m15[m15['fvg'] != 0].iterrows():
        fvg_dir = int(fvg['fvg'])
        fvg_time = fvg['time']
        fvg_top = float(fvg['fvg_top'])
        fvg_bottom = float(fvg['fvg_bottom'])

        m5_after = m5[m5['time'] > fvg_time]
        if m5_after.empty:
            continue

        # список фракталів (тільки реальні: !=0 і не NaN)
        fracs = m5_after[(m5_after['fractal'] != 0) & (~m5_after['fractal'].isna())]
        if len(fracs) < 2:
            continue

        # позиційні індекси
        frac_positions = list(fracs.index)

        # перебираємо сусідні пари (p1, p2)
        for i in range(len(frac_positions) - 1):
            p1 = frac_positions[i]
            p2 = frac_positions[i + 1]
            r1 = m5.loc[p1]
            r2 = m5.loc[p2]

            if fvg_dir == 1:
                # long: f1 == 1 та f1.price > fvg_top
                #       f2 == -1 та f2.price < fvg_top і > fvg_bottom
                cond_pair = (r1['fractal'] == 1) and (float(r1['fractal_level']) > fvg_top) \
                            and (r2['fractal'] == -1) and (float(r2['fractal_level']) < fvg_top) and (float(r2['fractal_level']) > fvg_bottom)
            else:
                # short: f1 == -1 та f1.price < fvg_top
                #        f2 == 1 та f2.price > fvg_top і < fvg_bottom
                cond_pair = (r1['fractal'] == -1) and (float(r1['fractal_level']) < fvg_bottom) \
                            and (r2['fractal'] == 1) and (float(r2['fractal_level']) > fvg_bottom) and (float(r2['fractal_level']) < fvg_top)

            if not cond_pair:
                continue

            # починаємо шукати BOS з наступної свічки після f2
            start_pos = p2
            if start_pos >= n_m5:
                break

            f1_price = float(r1['fractal_level'])
            bos_pos = None

            end_pos = n_m5 if max_lookahead is None else min(n_m5, start_pos + int(max_lookahead))

            for pos in range(start_pos, end_pos):
                close = float(m5.at[pos, 'close'])
                if fvg_dir == 1:
                    cond = close > f1_price   # long: close вище f1
                elif fvg_dir == -1:
                    cond = close < f1_price   # short: close нижче f1
                else:
                    return None

                if cond:
                    bos_pos = pos
                    break

            if bos_pos is not None:
                sig = {
                    'fvg_dir': fvg_dir,
                    'fvg_time': fvg_time,
                    'fvg_top': fvg_top,
                    'fvg_bottom': fvg_bottom,
                    'f1_pos': int(p1),
                    'f1_time': r1['time'],
                    'f1_price': float(r1['fractal_level']),
                    'f2_pos': int(p2),
                    'f2_time': r2['time'],
                    'f2_price': float(r2['fractal_level']),
                    'bos_pos': int(bos_pos),
                    'bos_time': m5.at[bos_pos, 'time'],
                    'bos_close': float(m5.at[bos_pos, 'close'])
                }
                signals.append(sig)

                m5.at[p1, 'is_f1'] = 1
                m5.at[p2, 'is_f2'] = 1
                m5.at[bos_pos, 'is_bos'] = 1
                m5.at[p1, 'source_fvg_time'] = fvg_time
                m5.at[p2, 'source_fvg_time'] = fvg_time
                m5.at[bos_pos, 'source_fvg_time'] = fvg_time
                
    signals_df = pd.DataFrame(signals)
    if '_orig_index' in m5.columns:
        m5_out = m5.set_index('_orig_index').sort_index()
    else:
        if debug:
            print("Warning: '_orig_index' not in m5.columns; returning positional-index m5")
        m5_out = m5

    return signals_df, m5_out

signals, m5_annot = find_fvg_fractal_bos_by_close(ohlc_m15, ohlc_m5, max_lookahead=len(ohlc_m5))

signals.tail()

,fvg_dir,fvg_time,fvg_top,fvg_bottom,f1_pos,f1_time,f1_price,f2_pos,f2_time,f2_price,bos_pos,bos_time,bos_close
552,1,2025-10-07 04:45:00,124417.0,124352.0,2379,2025-10-07 05:30:00,124508.3,2380,2025-10-07 05:35:00,124357.2,2380,2025-10-07 05:35:00,124509.8
553,-1,2025-10-07 06:30:00,124219.6,123949.5,2398,2025-10-07 07:05:00,123817.9,2399,2025-10-07 07:10:00,124000.8,2399,2025-10-07 07:10:00,123753.4
554,-1,2025-10-07 06:30:00,124219.6,123949.5,2426,2025-10-07 09:25:00,123941.6,2429,2025-10-07 09:40:00,124200.0,2477,2025-10-07 13:40:00,123734.2
555,-1,2025-10-07 06:30:00,124219.6,123949.5,2479,2025-10-07 13:50:00,123316.9,2482,2025-10-07 14:05:00,123962.8,2482,2025-10-07 14:05:00,123023.0
556,-1,2025-10-07 13:45:00,124569.5,123940.3,2479,2025-10-07 13:50:00,123316.9,2482,2025-10-07 14:05:00,123962.8,2482,2025-10-07 14:05:00,123023.0


In [22]:
from main import qty_from_risk

def qty_from_risk(entry_price, stop_loss, risk_usd=1000):
    dist = abs(entry_price - stop_loss)
    if dist <= 0:
        return 0
    qty = risk_usd / dist
    return round(qty, 3)

def calc_pnl(entry_price, exit_price, stop_loss, risk_usd=1000, direction=1):
    # 1. визначаємо кількість (одиниці активу)
    qty = qty_from_risk(entry_price, stop_loss, risk_usd)

    # 2. обчислюємо різницю в ціні з урахуванням напряму
    price_diff = (exit_price - entry_price) * direction

    # 3. PnL у доларах
    pnl = qty * price_diff
    return pnl, qty


def run_backtest_signals(signals_df, m5_df, rr=2.5, max_lookahead=None,
                         conservative_same_candle=True, strict_entry_candle=False):
    """
    Як раніше, але додає перевірку: якщо TP/SL були торкнуті **до** фактичної
    свічки входу (у проміжку від BOS до entry_idx-1), сигнал пропускається.
    strict_entry_candle: якщо True - і в самій свічці входу є торк TP/SL -> пропускаємо
    (бо не можемо відрізнити порядок подій).
    """
    if signals_df is None or len(signals_df) == 0:
        return pd.DataFrame()

    m5 = m5_df.copy()
    m5['time'] = pd.to_datetime(m5['time'])
    m5 = m5.sort_values('time').reset_index(drop=True)

    trades = []
    last_idx = len(m5) - 1

    for _, sig in signals_df.iterrows():
        try:
            fvg_dir = int(sig.get('fvg_dir', 0))
        except Exception:
            continue
        if fvg_dir == 0:
            continue

        f1_price = float(sig['f1_price'])
        f2_price = float(sig['f2_price'])

        # базовий старт після BOS
        if 'bos_time' in sig and pd.notnull(sig['bos_time']):
            bos_time = pd.to_datetime(sig['bos_time'])
            start_loc = int(m5['time'].searchsorted(bos_time, side='right'))
        elif 'bos_pos' in sig and not pd.isna(sig['bos_pos']):
            pos = int(sig['bos_pos'])
            start_loc = pos + 1
        else:
            continue

        if start_loc >= len(m5):
            continue

        # розрахуємо TP/SL заздалегідь (вхід по f1_price)
        direction = 1 if fvg_dir == 1 else -1
        entry_price = f1_price
        sl = f2_price
        if direction == 1:
            risk = entry_price - sl
            tp = entry_price + risk * rr
        else:
            risk = sl - entry_price
            tp = entry_price - risk * rr

        if abs(risk) < 1e-12:
            continue

        # межа пошуку entry (lookahead)
        if max_lookahead is not None:
            search_end = min(last_idx, start_loc + int(max_lookahead) - 1)
        else:
            search_end = last_idx

        entry_idx = None

        # перебираємо свічки після BOS, шукаємо першу що торкнула f1
        for idx in range(start_loc, search_end + 1):
            row = m5.loc[idx]
            high = float(row['high'])
            low = float(row['low'])

            if low <= f1_price <= high:
                # candidate entry found at idx — але спочатку перевіримо, чи не було TP/SL раніше
                prior_touched = False
                if idx > start_loc:
                    prior_slice = m5.loc[start_loc:idx-1]
                    # перевіряємо, чи будь-яка свічка в prior_slice зачепила TP або SL
                    if direction == 1:
                        # long: prior touched tp якщо high >= tp; prior touched sl якщо low <= sl
                        if (prior_slice['high'].astype(float) >= tp).any() or (prior_slice['low'].astype(float) <= sl).any():
                            prior_touched = True
                    else:
                        # short: prior touched tp якщо low <= tp; prior touched sl якщо high >= sl
                        if (prior_slice['low'].astype(float) <= tp).any() or (prior_slice['high'].astype(float) >= sl).any():
                            prior_touched = True

                if prior_touched:
                    # якщо до entry вже торкнулися TP/SL — цей сигнал пропускаємо повністю
                    entry_idx = None
                    break  # немає сенсу шукати далі по цим signal'у
                else:
                    # опційно: якщо в самій entry-candle також торкнули TP/SL і strict_entry_candle==True -> пропускаємо
                    entry_candle_touched = False
                    if strict_entry_candle:
                        if direction == 1:
                            if (high >= tp) or (low <= sl):
                                entry_candle_touched = True
                        else:
                            if (low <= tp) or (high >= sl):
                                entry_candle_touched = True
                        if entry_candle_touched:
                            entry_idx = None
                            break

                    # якщо пройшли перевірки — підтверджуємо entry_idx
                    entry_idx = idx
                    break

        if entry_idx is None:
            # ні одна свічка після BOS не дала валідного входу (або була попередня зачепа TP/SL)
            continue

        # симуляція закриття як раніше, починаючи з entry_idx
        if max_lookahead is not None:
            end_idx = min(last_idx, entry_idx + int(max_lookahead) - 1)
        else:
            end_idx = last_idx

        exit_idx = None
        exit_price = None
        result = None
        pnl = None

        for idx in range(entry_idx, end_idx + 1):
            row = m5.loc[idx]
            high = float(row['high'])
            low = float(row['low'])

            if direction == 1:
                touched_sl = low <= sl
                touched_tp = high >= tp

                if touched_sl and touched_tp:
                    if conservative_same_candle:
                        exit_idx = idx; exit_price = sl; result = 'loss'; pnl = sl - entry_price; break
                    else:
                        exit_idx = idx; exit_price = tp; result = 'win'; pnl = tp - entry_price; break
                elif touched_sl:
                    exit_idx = idx; exit_price = sl; result = 'loss'; pnl = sl - entry_price; break
                elif touched_tp:
                    exit_idx = idx; exit_price = tp; result = 'win'; pnl = tp - entry_price; break
            else:
                touched_sl = high >= sl
                touched_tp = low <= tp

                if touched_sl and touched_tp:
                    if conservative_same_candle:
                        exit_idx = idx; exit_price = sl; result = 'loss'; pnl = entry_price - sl; break
                    else:
                        exit_idx = idx; exit_price = tp; result = 'win'; pnl = entry_price - tp; break
                elif touched_sl:
                    exit_idx = idx; exit_price = sl; result = 'loss'; pnl = entry_price - sl; break
                elif touched_tp:
                    exit_idx = idx; exit_price = tp; result = 'win'; pnl = entry_price - tp; break

        if exit_idx is None:
            exit_idx = end_idx
            exit_price = float(m5.loc[end_idx, 'close'])
            pnl = (exit_price - entry_price) if direction == 1 else (entry_price - exit_price)
            result = 'timeout'


        qty = qty_from_risk(entry_price, sl, risk_usd=1000)
        pnl = (exit_price - entry_price) * direction * qty

        trades.append({
            'signal_index': sig.name if sig.name is not None else None,
            'direction': direction,
            'fvg_time': pd.to_datetime(sig.get('fvg_time', pd.NaT)),
            'fvg_top': float(sig.get('fvg_top', np.nan)),
            'fvg_bottom': float(sig.get('fvg_bottom', np.nan)),
            'f1_time': pd.to_datetime(sig.get('f1_time', pd.NaT)),
            'f1_price': float(sig.get('f1_price', np.nan)),
            'f2_time': pd.to_datetime(sig.get('f2_time', pd.NaT)),
            'f2_price': float(sig.get('f2_price', np.nan)),
            'entry_time': m5.loc[entry_idx, 'time'],
            'entry_index': int(entry_idx),
            'entry_price': entry_price,
            'sl': sl,
            'tp': tp,
            'exit_time': m5.loc[int(exit_idx), 'time'],
            'exit_index': int(exit_idx),
            'exit_price': exit_price,
            'result': result,
            'pnl': round(pnl, 0)
        })

    trades_df = pd.DataFrame(trades)

    return trades_df

trades = run_backtest_signals(signals, m5_annot, rr=2.5, max_lookahead=20, conservative_same_candle=True)


In [23]:
trades.tail()

,signal_index,direction,fvg_time,fvg_top,fvg_bottom,f1_time,f1_price,f2_time,f2_price,entry_time,entry_index,entry_price,sl,tp,exit_time,exit_index,exit_price,result,pnl
449,547,-1,2025-10-06 23:15:00,124829.5,124811.8,2025-10-06 23:25:00,124604.9,2025-10-06 23:35:00,124813.8,2025-10-07 00:25:00,2318,124604.9,124813.8,124082.65,2025-10-07 00:40:00,2321,124813.8,loss,-1000.0
450,549,1,2025-10-07 04:30:00,124324.0,124247.0,2025-10-07 05:15:00,124469.7,2025-10-07 05:20:00,124290.6,2025-10-07 05:40:00,2381,124469.7,124290.6,124917.45,2025-10-07 05:50:00,2383,124290.6,loss,-1000.0
451,550,1,2025-10-07 04:30:00,124324.0,124247.0,2025-10-07 12:10:00,124462.4,2025-10-07 12:20:00,124300.0,2025-10-07 12:40:00,2465,124462.4,124300.0,124868.40,2025-10-07 12:45:00,2466,124868.4,win,2500.0
452,552,1,2025-10-07 04:45:00,124417.0,124352.0,2025-10-07 05:30:00,124508.3,2025-10-07 05:35:00,124357.2,2025-10-07 05:40:00,2381,124508.3,124357.2,124886.05,2025-10-07 05:45:00,2382,124357.2,loss,-1000.0
453,554,-1,2025-10-07 06:30:00,124219.6,123949.5,2025-10-07 09:25:00,123941.6,2025-10-07 09:40:00,124200.0,2025-10-07 14:00:00,2481,123941.6,124200.0,123295.60,2025-10-07 14:05:00,2482,123295.6,win,2500.0


In [24]:
from Utils.metrics import calculate_metrics

all_results = []
trades_data = run_backtest_signals(signals, m5_annot, rr=2.5, max_lookahead=20, conservative_same_candle=True)
trades_data.to_csv('trades_data.csv', index=False)
profits = trades_data['pnl']
results_metrics = calculate_metrics(profits)
all_results.append(results_metrics)

all_results = pd.concat(all_results, axis=0)
all_results.to_csv('all_results.csv', index=False)
all_results


,Total Trades,Total Net Profit,Average Profit,Win rate,Average Win,Average Loss,Profit Factor,Max Consecutive Losses,Max Drawdown
0,454,59051.0,130.07,0.44,1391.89,-845.88,1.27,10,30892.0
